<a href="https://colab.research.google.com/github/zeton24/gsn_iot_anomalies_detection/blob/main/data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
#potrzebne, zeby miec wersje chociaz 0.20.23
!pip install polars --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.2/27.2 MB 10.3 MB/s eta 0:00:00
  Attempting uninstall: polars
    Found existing installation: polars 0.20.2
    Uninstalling polars-0.20.2:
      Successfully uninstalled polars-0.20.2


In [2]:
! pip install pytorch-lightning

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 802.2/802.2 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 15.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-many

In [3]:
import os
import random
import gdown

import torch
import pytorch_lightning as pl
from torch.utils.data import random_split, DataLoader
from torchvision.transforms import Resize, Compose, ToTensor, Normalize


In [4]:
def to_one_hot(category, n_classes):
  result = torch.zeros(n_classes, 1)
  result[category] = 1
  return result

In [10]:
import polars as pol

class IoTDataset:
    def __init__(self, file_path: str, binary_classification=False):
        self.data = pol.read_csv(file_path)
        self.binary_classification = binary_classification
        if binary_classification:
          mapping = {"Anomaly": 1, "Normal": 0}
          self.labels = self.data['Label'].replace(mapping, return_dtype=pol.UInt8)
        else:
          mapping = {'Normal': 0, 'Mirai': 1, 'DoS': 2, 'Scan': 3, 'MITM ARP Spoofing': 4}
          self.labels = self.data['Cat'].replace(mapping, return_dtype=pol.UInt8)
        self.data = self.data[:, :-2]
        self.length = self.__len__()

    def __len__(self):
        return self.data.select(pol.count()).item()

    def __getitem__(self, index):
        data = self.data[index].to_torch()
        label = self.labels[index].item() if self.binary_classification else to_one_hot(self.labels[index], 16)
        return data, label


In [11]:
iot = IoTDataset('/content/iot-intrusion_with_headers.csv')

<ipython-input-10-bc51675aea77>:17: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  return self.data.select(pol.count()).item()


In [13]:
class IoTDataModule(pl.LightningDataModule):
    def __init__(self, batch_size, file_id: str, file_name: str, num_classes: int):
        super().__init__()
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.file_id = file_id
        self.file_name = file_name

    def prepare_data(self):
      if not os.path.exists(self.file_name):
        gdown.download(id=self.file_id, output=self.file_name)

    def setup(self, stage=None):
        # train/val
        if stage == 'fit' or stage is None:
            binary = self.num_classess == 2
            dataset = IoTDataset(file_path=self.file_name, binary_classification=binary)
            train_dataset_size = int(len(dataset) * 0.9)
            self.train_dataset, self.val_dataset = random_split(dataset, [train_dataset_size, len(dataset) - train_dataset_size])
        # test
        if stage == 'test' or stage is None:
            self.test_dataset = []

    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, shuffle=True)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size)


In [ ]:
intrusion_csv = '1k8RqOM7hBvL8uomcKnnLr5HasYRWnYeW'

In [14]:
dm = IoTDataModule(batch_size=16, file_id=intrusion_csv, file_name='iot-intrusion_with_headers.csv', num_classes=16)
dm.prepare_data()
dm.setup()

<ipython-input-10-bc51675aea77>:17: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
  return self.data.select(pol.count()).item()


In [15]:
loader = dm.train_dataloader()

In [17]:
for x,y in loader:
  print(len(y))
  print(x)
  break

16
tensor([[[126.0000,   1.0000,   1.0000,  ...,   0.0000, 126.0000, 126.0000]],

        [[ 84.0000,   0.0000,   2.0000,  ...,   0.0000,  84.0000,  84.0000]],

        [[252.0000,   2.0000,   1.0000,  ...,  12.7279, 135.0000, 117.0000]],

        ...,

        [[149.0000,   2.0000,   1.0000,  ...,   0.7071,  75.0000,  74.0000]],

        [[157.0000,   0.0000,   3.0000,  ...,   6.3640,  83.0000,  74.0000]],

        [[ 77.0000,   1.0000,   1.0000,  ...,   0.0000,  77.0000,  77.0000]]],
       dtype=torch.float64)
